In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [8]:
# key_1
# client_credentials_manager = SpotifyClientCredentials(client_id='e48676801c514fd0a6202eb890eaa0f1', client_secret='발급받은 키')

# key_2
# client_credentials_manager = SpotifyClientCredentials(client_id='5022af45601e4149a0881fbbe27eef62', client_secret='발급받은 키')

# key_3
client_credentials_manager = SpotifyClientCredentials(client_id='f588ee129cec4da4a194ece6210ffab6', client_secret='발급받은 키')

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# API 데이터 요청

In [9]:
# 2021년 음악 1000개
track_id =[]
track_name = []
artist_names =[]
track_popularity =[]
track_sampl =[]
artist_id =[]
album_id = []

artist_id_list =[]


for i in range(0,1000,50):
    # 2021년도 음악정보 받아오기
    track_results = sp.search(q='year:2021', type='track', limit=50, offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_names_buff =[]
        artist_id_buff = []
        
        # 각 음악의 아티스트 정보를 저장
        for j in range(len(t['artists'])):
            artist_names_buff.append(t['artists'][j]['name'])
            artist_id_buff.append(t['artists'][j]['id'])
            artist_id_list.append(t['artists'][j]['id'])
        artist_names.append(artist_names_buff)
        artist_id.append(artist_id_buff)
        
        # 트랙의 나머지 정보를 저장
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        album_id.append(t['album']['id'])
        track_sampl.append(t['preview_url'])

In [10]:
# album_id로 엘범을 불러온 뒤 그 엘범의 tracks_id를 얻음
album_name =[]
album_artists =[]
album_artists_id =[]
album_total_tracks =[]
album_tracks =[]
album_tracks_id =[]
album_popularity =[]

album_artists_id_list =[]
album_tracks_id_list =[]

batch_size = 20
album_id_setlist = list(set(album_id))

# 엘범정보를 불러와 저장
for i in range(0, len(album_id_setlist), batch_size):
    # batch_size만큼 엘범정보를 불러옴
    batch_album_ids = album_id_setlist[i:i+batch_size]
    albums = sp.albums(batch_album_ids)['albums']
    
    # 앨범 정보 처리
    for album in albums:
        album_name.append(album['name'])
        album_artists_buff =[]
        album_artists_id_buff =[]
        
        # 엘범의 아티스트 정보를 저장
        for i in range(len(album['artists'])):
            album_artists_buff.append(album['artists'][i]['name'])
            album_artists_id_buff.append(album['artists'][i]['id'])
            album_artists_id_list.append(album['artists'][i]['id'])
        album_artists.append(album_artists_buff)
        album_artists_id.append(album_artists_id_buff)

        #엘범의 트랙수를 가져옴
        total = album['total_tracks']
        album_total_tracks.append(total)
        album_tracks_buff =[]
        album_tracks_id_buff =[]
        
        # 엘범에 수록된 트랙의 정보를 저장
        for j in range(total):
            album_tracks_buff.append(album['tracks']['items'][j]['name'])
            album_tracks_id_buff.append(album['tracks']['items'][j]['id'])
            album_tracks_id_list.append(album['tracks']['items'][j]['id'])
        album_tracks.append(album_tracks_buff)
        album_tracks_id.append(album_tracks_id_buff)
        
        album_popularity.append(album['popularity'])

In [11]:
# track_id의 중복값과 기존값을 제거
add_track_id = list(set(album_tracks_id_list) - set(track_id))

In [12]:
# 추가로 얻은 track_id를 더 불러옴
track_id_add =[]
track_name_add = []
artist_names_add =[]
track_popularity_add =[]
track_sampl_add =[]
artist_id_add =[]
album_id_add = []

artist_id_list_add =[]
artist_names_list_add =[]

batch_size = 20

# 엘범에서 얻은 트랙의 ID로 더 불러오기
for i in range(0, len(add_track_id), batch_size):
    batch_album_ids = add_track_id[i:i+batch_size]
    tracks = sp.tracks(batch_album_ids)['tracks']
    for track in tracks:
        artist_names_buff =[]
        artist_id_buff = []
        # 트랙의 아티스트 정보
        for j in range(len(track['artists'])):
            artist_names_buff.append(track['artists'][j]['name'])
            artist_id_buff.append(track['artists'][j]['id'])
            artist_names_list_add.append(track['artists'][j]['name'])
            artist_id_list_add.append(track['artists'][j]['id'])
        artist_names_add.append(artist_names_buff)
        artist_id_add.append(artist_id_buff)
        
        # 트랙의 나머지 정보
        track_name_add.append(track['name'])
        track_id_add.append(track['id'])
        track_popularity_add.append(track['popularity'])
        album_id_add.append(track['album']['id'])
        track_sampl_add.append(track['preview_url'])

In [13]:
# 추가 데이터 넣어주기
track_id.extend(add_track_id)
track_name.extend(track_name_add)
artist_names.extend(artist_names_add)
track_popularity.extend(track_popularity_add)
track_sampl.extend(track_sampl_add)
artist_id.extend(artist_id_add)
album_id.extend(album_id_add)

artist_id_list.extend(artist_id_list_add)

In [14]:
# 아티스트 정보 받기
artist_name =[]
artist_popularity = []
artist_genres = []
artist_followers =[]

batch_size = 20
album_id_setlist = list(set(album_id))

# 트랙정보에 있는 아티스트 정보를 저장
for i in range(0, len(list(set(artist_id_list))), batch_size):
    batch_album_ids = list(set(artist_id_list))[i:i+batch_size]
    artists = sp.artists(batch_album_ids)['artists']
    for artist in artists:
        artist_name.append(artist['name'])
        artist_popularity.append(artist['popularity'])
        artist_genres.append(artist['genres'])
        artist_followers.append(artist['followers']['total'])

In [15]:
# 트렉 상세정보 받기
track_features = []
track_details_df = pd.DataFrame(columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'url', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

batch_size = 20

# 트랙이 가지고 있는 특징 데이터를 저장
for i in range(0, len(track_id), batch_size):
    batch_track_ids = track_id[i:i + batch_size]
    batch_af = sp.audio_features(batch_track_ids)
    track_features.append(batch_af)
    
# 데이터 프레임으로 저장함
for item in track_features:
    for feat in item:
        track_details_df = track_details_df.append(feat, ignore_index=True)
track_details_df.head()

C:\Users\Medici\AppData\Local\Temp\ipykernel_17540\2017581899.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  track_details_df = track_details_df.append(feat, ignore_index=True)
C:\Users\Medici\AppData\Local\Temp\ipykernel_17540\2017581899.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  track_details_df = track_details_df.append(feat, ignore_index=True)
C:\Users\Medici\AppData\Local\Temp\ipykernel_17540\2017581899.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  track_details_df = track_details_df.append(feat, ignore_index=True)
C:\Users\Medici\AppData\Local\Temp\ipykernel_17540\2017581899.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use panda

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,url,track_href,analysis_url,duration_ms,time_signature,uri
0,0.505,0.657,11,-5.240,0,0.0318,0.3730,0.001070,0.1260,0.252,196.000,audio_features,3cBsEDNhFI9E82vPj3kvi3,NaN,https://api.spotify.com/v1/tracks/3cBsEDNhFI9E...,https://api.spotify.com/v1/audio-analysis/3cBs...,178520,3,spotify:track:3cBsEDNhFI9E82vPj3kvi3
1,0.807,0.595,0,-7.190,1,0.0325,0.3100,0.000024,0.0735,0.392,92.491,audio_features,2VzCjpKvPB1l1tqLndtAQa,NaN,https://api.spotify.com/v1/tracks/2VzCjpKvPB1l...,https://api.spotify.com/v1/audio-analysis/2VzC...,239845,4,spotify:track:2VzCjpKvPB1l1tqLndtAQa
2,0.645,0.534,6,-10.800,0,0.0479,0.1570,0.000000,0.0863,0.463,105.020,audio_features,3azJifCSqg9fRij2yKIbWz,NaN,https://api.spotify.com/v1/tracks/3azJifCSqg9f...,https://api.spotify.com/v1/audio-analysis/3azJ...,226467,4,spotify:track:3azJifCSqg9fRij2yKIbWz
3,0.849,0.424,5,-9.579,0,0.3240,0.0635,0.000000,0.0834,0.153,145.887,audio_features,2BcMwX1MPV6ZHP4tUT9uq6,NaN,https://api.spotify.com/v1/tracks/2BcMwX1MPV6Z...,https://api.spotify.com/v1/audio-analysis/2BcM...,242966,4,spotify:track:2BcMwX1MPV6ZHP4tUT9uq6
4,0.620,0.869,6,-5.479,1,0.0462,0.6150,0.000000,0.1310,0.597,126.019,audio_features,5OELUCYgOHKFAvCERnAvfS,NaN,https://api.spotify.com/v1/tracks/5OELUCYgOHKF...,https://api.spotify.com/v1/audio-analysis/5OEL...,216573,4,spotify:track:5OELUCYgOHKFAvCERnAvfS


# 데이터프레임 만들기

In [16]:
# 트렉 데이터 프레임
track_df = pd.DataFrame({'track_id' : track_id, 'track_name' : track_name, 'artist_names': artist_names, 'track_popularity' : track_popularity, 'track_sampl' : track_sampl, 'artist_id': artist_id, 'album_id' : album_id})
print(track_df.shape)
track_df.head()

(4348, 7)


,track_id,track_name,artist_names,track_popularity,track_sampl,artist_id,album_id
0,3cBsEDNhFI9E82vPj3kvi3,Wasted On You,[Morgan Wallen],86,https://p.scdn.co/mp3-preview/185a44ab4036da72...,[4oUHIQIBe0LHzYfvXNW4QM],6JlCkqkqobGirPsaleJpFr
1,2VzCjpKvPB1l1tqLndtAQa,Por las Noches,[Peso Pluma],93,https://p.scdn.co/mp3-preview/ba2f667c373e2d12...,[12GqGscKJx3aE4t07u7eVZ],6USJaTtVIy0YH5hhzOWVHe
2,3azJifCSqg9fRij2yKIbWz,The Color Violet,[Tory Lanez],88,https://p.scdn.co/mp3-preview/1d7c627bf5493281...,[2jku7tDXc6XoB6MO2hFuqg],5Gm2XKBgnlzd6qTi7LE1z2
3,2BcMwX1MPV6ZHP4tUT9uq6,Knife Talk (with 21 Savage ft. Project Pat),"[Drake, 21 Savage, Project Pat]",84,https://p.scdn.co/mp3-preview/85d634aaeb6d71c2...,"[3TVXtAsR1Inumwj472S9r4, 1URnnhqYAYcrqrcwql10f...",3SpBlxme9WbeQdI9kx7KAV
4,5OELUCYgOHKFAvCERnAvfS,More Than My Hometown,[Morgan Wallen],81,https://p.scdn.co/mp3-preview/7db06b5c21f07696...,[4oUHIQIBe0LHzYfvXNW4QM],6JlCkqkqobGirPsaleJpFr


In [17]:
# 아티스트 데이터 프레임
artist_df = pd.DataFrame({'artist_id' : list(set(artist_id_list)),'artist_name' : artist_name, 'artist_popularity' : artist_popularity, 'artist_genres' : artist_genres, 'artist_followers' : artist_followers})
print(artist_df.shape)
artist_df.head()

(1163, 5)


,artist_id,artist_name,artist_popularity,artist_genres,artist_followers
0,6RHTUrRF63xao58xh9FXYJ,IVE,79,"[k-pop, k-pop girl group]",2131253
1,7dOvC6mtaZ6jvuReZwMkkM,AO Meally,19,[],7
2,3jK9MiCrA42lLAdMGUZpwa,Anderson .Paak,78,"[escape room, hip hop, indie soul, neo soul]",2345238
3,37BIBvfliXJpSgn10yQUJM,Mr. Naisgai,58,[],2277
4,0mA4dkNGiN4fqTBi2SLlAv,Edgardo Nuñez,73,"[corrido, corridos tumbados, sad sierreno, sie...",545986


In [18]:
# 엘범 데이터 프레임
album_df = pd.DataFrame({'album_id' : list(set(album_id)), 'album_name' : album_name, 'album_artists' : album_artists, 'album_artists_id' : album_artists_id, 'album_total_tracks' : album_total_tracks, 'album_tracks' : album_tracks, 'album_tracks_id': album_tracks_id, 'album_popularity' : album_popularity})
print(album_df.shape)
album_df.head()

(644, 8)


,album_id,album_name,album_artists,album_artists_id,album_total_tracks,album_tracks,album_tracks_id,album_popularity
0,3cLQ49Ll3nGRkw3HSSk92K,Expensive Pain,[Meek Mill],[20sxb77xiYeusSH8cVdatc],18,"[Intro (Hate on Me), Outside (100 MPH), On My ...","[4s7TIubkdsgdtDEDFEAYVL, 67lvhLT01Fq3GEIrt5RpP...",66
1,5eCmZVM5Kspu2B1cyVdSqq,Ashlyn,[Ashe],[6P5NO5hzJbuOqSdyPB7SJM],14,"[Till Forever Falls Apart, I'm Fine, Love Is N...","[4x5YzxjrF9wvIIXJV7TveC, 4mBzAVxr3Mias8T4zWF52...",76
2,6s84u2TUpR3wdUv4NgKA2j,SOUR,[Olivia Rodrigo],[1McMsnEElThX1knmY4oliG],11,"[brutal, traitor, drivers license, 1 step forw...","[6SRsiMl7w1USE4mFqrOhHC, 5CZ40GBx1sQ9agT82CLQC...",93
3,35qVMfUfBN6q2xzm9rZn5a,INSIDE,[Bo Burnham],[2Waw2sSbqvAwK8NwACNjVo],21,"[Content, Comedy, FaceTime with my Mom (Tonigh...","[4GDL4Rl9ov0OctWIm6ZkoR, 50MhgV5PrQmCSuEELaYp9...",78
4,576UTio7xgY7YUxrHMzxeI,F*CK LOVE 3+: OVER YOU,[The Kid LAROI],[2tIP7SsRs7vjIcLrU85W8J],35,"[OVER YOU, NOT SOBER (feat. Polo G & Stunna Ga...","[23JVRF8F2C1TV2QDDNn213, 29t9XYOV65fFXiVzFFJgH...",86


In [19]:
# 트렉 디테일 데이터 프레임
track_details_df = track_details_df.drop(['key', 'mode','type','url','track_href', 'analysis_url','time_signature', 'uri'], axis=1)

new_order = ['id', 'danceability', 'energy','loudness','speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']
track_details_df = track_details_df.reindex(columns=new_order)
track_details_df = track_details_df.rename(columns={'id': 'track_id'})
track_details_df.head()

,track_id,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,3cBsEDNhFI9E82vPj3kvi3,0.505,0.657,-5.240,0.0318,0.3730,0.001070,0.1260,0.252,196.000,178520
1,2VzCjpKvPB1l1tqLndtAQa,0.807,0.595,-7.190,0.0325,0.3100,0.000024,0.0735,0.392,92.491,239845
2,3azJifCSqg9fRij2yKIbWz,0.645,0.534,-10.800,0.0479,0.1570,0.000000,0.0863,0.463,105.020,226467
3,2BcMwX1MPV6ZHP4tUT9uq6,0.849,0.424,-9.579,0.3240,0.0635,0.000000,0.0834,0.153,145.887,242966
4,5OELUCYgOHKFAvCERnAvfS,0.620,0.869,-5.479,0.0462,0.6150,0.000000,0.1310,0.597,126.019,216573


In [20]:
# 각 데이터 저장
path = r'C:\Users\Medici\Vscode\SQL\data'
track_df.to_csv(path + r'ori_track.csv', index=False, encoding='utf-8')
artist_df.to_csv(path + r'ori_artist.csv', index=False, encoding='utf-8')
album_df.to_csv(path + r'ori_album.csv', index=False, encoding='utf-8')
track_details_df.to_csv(path + r'ori_track_details.csv', index=False, encoding='utf-8')

# 데이터 평탄화 & 테이블 생성

album to artists

In [16]:
# 엘범과 아티스트 데이터의 정규화 진행
album_artists_list1 = []
for t in zip(album_id_setlist, album_artists):
    album_artists_list1.append({"album_id": t[0], "album_artists": t[1]} )
    
album_artists_list2 = []
for t in zip(album_id_setlist, album_artists_id):
    album_artists_list2.append({"album_id": t[0], "album_artists_id": t[1]} )

In [17]:
# json형식의 데이터 평탄화 진행
album_artists_df1 = pd.json_normalize(album_artists_list1, record_path=("album_artists"), meta=["album_id"])
album_artists_df2 = pd.json_normalize(album_artists_list2, record_path=("album_artists_id"), meta=["album_id"])

album_artists_df1 = album_artists_df1.rename(columns={0:'album_artists'})
album_artists_df2 = album_artists_df2.rename(columns={0:'album_artists_id'})

In [18]:
album_artists_df = album_artists_df1.join(album_artists_df2['album_artists_id'])
album_artists_df.head()

,album_artists,album_id,album_artists_id
0,Lil Tecca,7nTvm6EgmFWPLMLxj6aksS,4Ga1P7PMIsmqEZqhYZQgDo
1,Calma Interiore,3dwBW5p65buVtiTmr2a0Wl,1OmJlIsNDkAVfdkmHbvQFM
2,Sturgill Simpson,5CigVuEyiAYaNYx2M2w4hh,3vDpQbGnzRbRVirXlfQagB
3,Brent Faiyaz,2M3IBMpOszCjnxwG1sWOqT,3tlXnStJ1fFhdScmQeLpuG
4,Avery Anna,0HZiTcB82OPdvbJ2Hny6bK,5rjz63sgMaAUAyBMyCUK5D


album to tracks

In [19]:
# 엘범과 트랙의 데이터의 정규화 진행
album_tracks_list1 = []
for t in zip(album_id_setlist, album_tracks):
    album_tracks_list1.append({"album_id": t[0], "album_tracks": t[1]} )

album_tracks_list2 = []
for t in zip(album_id_setlist, album_tracks_id):
    album_tracks_list2.append({"album_id": t[0], "album_tracks_id": t[1]} )

In [20]:
# json형식의 데이터 평탄화 진행
album_tracks_df1 = pd.json_normalize(album_tracks_list1, record_path=("album_tracks"), meta=["album_id"])
album_tracks_df2 = pd.json_normalize(album_tracks_list2, record_path=("album_tracks_id"), meta=["album_id"])

album_tracks_df1 = album_tracks_df1.rename(columns={0:'album_tracks'})
album_tracks_df2 = album_tracks_df2.rename(columns={0:'album_tracks_id'})

In [21]:
album_tracks_df = album_tracks_df1.join(album_tracks_df2['album_tracks_id'])
album_tracks_df.head()

,album_tracks,album_id,album_tracks_id
0,MONEY ON ME,7nTvm6EgmFWPLMLxj6aksS,668vMRYpYfpLsOvWc73umT
1,REPEAT IT (with Gunna),7nTvm6EgmFWPLMLxj6aksS,4s6vwFgPdCtidpEw8KuQT6
2,NEVER LEFT,7nTvm6EgmFWPLMLxj6aksS,0soNGRgFn7aq1T4LXtFDMb
3,CAUTION,7nTvm6EgmFWPLMLxj6aksS,7J34pk34Aw4U5J7cL5XO98
4,SEASIDE (feat. iann dior),7nTvm6EgmFWPLMLxj6aksS,45bXTH6SuR2MKjypu4ksNz


track to artist

In [22]:
# 트랙과 아티스트의 데이터의 정규화 진행
track_artist_list1 = []
for t in zip(track_id, artist_id):
    track_artist_list1.append({"track_id": t[0], "artist_id": t[1]} )
    
track_artist_list2 = []
for t in zip(track_id, artist_names):
    track_artist_list2.append({"track_id": t[0], "artist_names": t[1]} )

In [23]:
# json형식의 데이터 평탄화 진행
track_artist_df1 = pd.json_normalize(track_artist_list1, record_path=("artist_id"), meta=["track_id"])
track_artist_df2 = pd.json_normalize(track_artist_list2, record_path=("artist_names"), meta=["track_id"])

track_artist_df1 = track_artist_df1.rename(columns={0:'artist_id'})
track_artist_df2 = track_artist_df2.rename(columns={0:'artist_names'})

In [24]:
track_artist_df = track_artist_df1.join(track_artist_df2['artist_names'])
track_artist_df.head()

,artist_id,track_id,artist_names
0,4oUHIQIBe0LHzYfvXNW4QM,3cBsEDNhFI9E82vPj3kvi3,Morgan Wallen
1,12GqGscKJx3aE4t07u7eVZ,2VzCjpKvPB1l1tqLndtAQa,Peso Pluma
2,2jku7tDXc6XoB6MO2hFuqg,3azJifCSqg9fRij2yKIbWz,Tory Lanez
3,3TVXtAsR1Inumwj472S9r4,2BcMwX1MPV6ZHP4tUT9uq6,Drake
4,1URnnhqYAYcrqrcwql10ft,2BcMwX1MPV6ZHP4tUT9uq6,21 Savage


artist to genres

In [25]:
# 아티스트와 장르의 데이터 정규화 및 평탄화
artist_id_list1 = list(set(artist_id_list))

artist_genres_list = []
for t in zip(artist_id_list1, artist_genres):
    artist_genres_list.append({"artist_id": t[0], "artist_genres": t[1]})
    
artist_genres_df = pd.json_normalize(artist_genres_list, record_path="artist_genres", meta=["artist_id"])
    
artist_genres_df = artist_genres_df.rename(columns={0:'artist_genres'})
artist_genres_df.head()

,artist_genres,artist_id
0,chicago rap,5g8mXUQcC8DVAHLwGGAIrJ
1,crunk,4Js9eYwAf9rypNtV8pNSw9
2,dirty south rap,4Js9eYwAf9rypNtV8pNSw9
3,north carolina hip hop,4Js9eYwAf9rypNtV8pNSw9
4,southern hip hop,4Js9eYwAf9rypNtV8pNSw9


In [35]:
# 각 데이터프레임 저장
path = r'C:\Users\Medici\Vscode\SQL\data'
artist_genres_df.to_csv(path + r'\ori_artist_genres.csv', index=False, encoding='utf-8')
track_artist_df.to_csv(path + r'\ori_track_artist.csv', index=False, encoding='utf-8')
album_tracks_df.to_csv(path + r'\ori_album_tracks.csv', index=False, encoding='utf-8')
album_artists_df.to_csv(path + r'\ori_album_artists.csv', index=False, encoding='utf-8')

In [42]:
# 테이블 파일 생성
path = r'C:\Users\Medici\Vscode\SQL\table'
track_table_df = track_df[['track_id', 'track_name', 'track_popularity', 'track_sampl', 'album_id']]
track_table_df.to_csv(path + r'\track_table.csv', index=False, encoding='utf-8')

artist_table_df = artist_df[['artist_id', 'artist_name', 'artist_popularity', 'artist_followers']]
artist_table_df.to_csv(path + r'\artist_table.csv', index=False, encoding='utf-8')

album_table_df = album_df[['album_id', 'album_name', 'album_total_tracks', 'album_popularity']]
album_table_df.to_csv(path + r'\album_table.csv', index=False, encoding='utf-8')

details_table_df = track_details_df
details_table_df.to_csv(path + r'\details_table.csv', index=False, encoding='utf-8')

# m : n
track_artist_table_df = track_artist_df[['artist_id', 'track_id']]
track_artist_table_df.to_csv(path + r'\track_artist_table.csv', index=False, encoding='utf-8')

# m : n
album_artists_table_df = album_artists_df[['album_id', 'album_artists_id']]
album_artists_table_df.to_csv(path + r'\album_artists_table.csv', index=False, encoding='utf-8')

# 다중속성
album_tracks_table_df = album_tracks_df[['album_id', 'album_tracks_id']]
album_tracks_table_df.to_csv(path + r'\album_tracks_table.csv', index=False, encoding='utf-8')

# 다중속성
artist_genres_table_df = artist_genres_df
artist_genres_table_df.to_csv(path + r'\artist_genres_table.csv', index=False, encoding='utf-8')


In [38]:
print(track_table_df.info())
print(artist_table_df.info())
print(album_table_df.info())
print(details_table_df.info())
print(track_artist_table_df.info())
print(album_artists_table_df.info())
print(album_tracks_table_df.info())
print(artist_genres_table_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4348 entries, 0 to 4347
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   track_id          4348 non-null   object
 1   track_name        4348 non-null   object
 2   track_popularity  4348 non-null   int64 
 3   track_sampl       3432 non-null   object
 4   album_id          4348 non-null   object
dtypes: int64(1), object(4)
memory usage: 170.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1163 entries, 0 to 1162
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   artist_id          1163 non-null   object
 1   artist_name        1163 non-null   object
 2   artist_popularity  1163 non-null   int64 
 3   artist_followers   1163 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 36.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 644 entries, 0 to 643

# MYsql에 데이터베이스 생성

In [2]:
# 테이블 데이터 불러오기
import pandas as pd
path = r'C:\Users\Medici\Vscode\SQL\table'
track_table_df = pd.read_csv(path + r'\track_table.csv', encoding='utf-8')
artist_table_df = pd.read_csv(path + r'\artist_table.csv', encoding='utf-8')
album_table_df = pd.read_csv(path + r'\album_table.csv', encoding='utf-8')
details_table_df = pd.read_csv(path + r'\details_table.csv', encoding='utf-8')
track_artist_table_df = pd.read_csv(path + r'\track_artist_table.csv', encoding='utf-8')
album_artists_table_df = pd.read_csv(path + r'\album_artists_table.csv', encoding='utf-8')
album_tracks_table_df = pd.read_csv(path + r'\album_tracks_table.csv', encoding='utf-8')
artist_genres_table_df = pd.read_csv(path + r'\artist_genres_table.csv', encoding='utf-8')

In [5]:
import mysql.connector
from mysql.connector import errorcode

# MySQL 서버에 연결

cnx = mysql.connector.connect(user='root', password='sqlds',
                                host='localhost')
cursor = cnx.cursor()

# 'spotify' 데이터베이스 생성
try:
    cursor.execute("CREATE DATABASE spotify")
    print("데이터베이스 'spotify'가 생성되었습니다.")
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_DB_CREATE_EXISTS:
        print("이미 'spotify' 데이터베이스가 존재합니다.")
    else:
        print("데이터베이스 생성 오류: {}".format(err))

# 'spotify' 데이터베이스 사용
cursor.execute("USE spotify")

# 테이블 생성
# 각 테이블의 생성 속성 또한 지정
tables = {}
tables['track'] = (
    "CREATE TABLE track ("
    "  track_id VARCHAR(255) NOT NULL,"
    "  track_name VARCHAR(255) NOT NULL,"
    "  track_popularity INT NOT NULL,"
    "  track_sampl VARCHAR(255),"
    "  album_id VARCHAR(255) NOT NULL,"
    "  PRIMARY KEY (track_id)"
    ") ENGINE=InnoDB")

tables['artist'] = (
    "CREATE TABLE artist ("
    "  artist_id VARCHAR(255) NOT NULL,"
    "  artist_name VARCHAR(255) NOT NULL,"
    "  artist_popularity INT NOT NULL,"
    "  artist_followers INT NOT NULL,"
    "  PRIMARY KEY (artist_id)"
    ") ENGINE=InnoDB")

tables['album'] = (
    "CREATE TABLE album ("
    "  album_id VARCHAR(255) NOT NULL,"
    "  album_name VARCHAR(255) NOT NULL,"
    "  album_total_tracks INT NOT NULL,"
    "  album_popularity INT NOT NULL,"
    "  PRIMARY KEY (album_id)"
    ") ENGINE=InnoDB")

tables['details'] = (
    "CREATE TABLE details ("
    "  track_id VARCHAR(255) NOT NULL,"
    "  danceability FLOAT NOT NULL,"
    "  energy FLOAT NOT NULL,"
    "  loudness FLOAT NOT NULL,"
    "  speechiness FLOAT NOT NULL,"
    "  acousticness FLOAT NOT NULL,"
    "  instrumentalness FLOAT NOT NULL,"
    "  liveness FLOAT NOT NULL,"
    "  valence FLOAT NOT NULL,"
    "  tempo FLOAT NOT NULL,"
    "  duration_ms VARCHAR(255) NOT NULL,"
    "  PRIMARY KEY (track_id)"
    ") ENGINE=InnoDB")

tables['track_artist'] = (
    "CREATE TABLE track_artist ("
    "  artist_id VARCHAR(255) NOT NULL,"
    "  track_id VARCHAR(255) NOT NULL,"
    "  PRIMARY KEY (artist_id, track_id)"
    ") ENGINE=InnoDB")

tables['album_artists'] = (
    "CREATE TABLE album_artists ("
    "  album_id VARCHAR(255) NOT NULL,"
    "  album_artists_id VARCHAR(255) NOT NULL,"
    "  PRIMARY KEY (album_id, album_artists_id)"
    ") ENGINE=InnoDB")

tables['album_tracks'] = (
    "CREATE TABLE album_tracks ("
    "  album_id VARCHAR(255) NOT NULL,"
    "  album_tracks_id VARCHAR(255) NOT NULL,"
    "  PRIMARY KEY (album_id, album_tracks_id)"
    ") ENGINE=InnoDB")

tables['artist_genres'] = (
    "CREATE TABLE artist_genres ("
    "  artist_genres VARCHAR(255) NOT NULL,"
    "  artist_id VARCHAR(255) NOT NULL,"
    "  PRIMARY KEY (artist_genres, artist_id)"
    ") ENGINE=InnoDB")

# 테이블 생성 확인
for table_name, table_definition in tables.items():
    try:
        cursor.execute(table_definition)
        print("테이블 '{}'이(가) 생성되었습니다.".format(table_name))
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("테이블 '{}'이(가) 이미 존재합니다.".format(table_name))
        else:
            print("테이블 생성 오류: {}".format(err))
            
# track_table 데이터 삽입
track_table_df = track_table_df.fillna('none')
track_data = track_table_df.values.tolist()
insert_query = "INSERT INTO track (track_id, track_name, track_popularity, track_sampl, album_id) VALUES (%s, %s, %s, %s, %s)"
cursor.executemany(insert_query, track_data)
cnx.commit()
print("track에 데이터가 삽입되었습니다.")

# artist_table 데이터 삽입
artist_data = artist_table_df.values.tolist()
insert_query = "INSERT INTO artist (artist_id, artist_name, artist_popularity, artist_followers) VALUES (%s, %s, %s, %s)"
cursor.executemany(insert_query, artist_data)
cnx.commit()
print("artist에 데이터가 삽입되었습니다.")

# album_table 데이터 삽입
album_data = album_table_df.values.tolist()
insert_query = "INSERT INTO album (album_id, album_name, album_total_tracks, album_popularity) VALUES (%s, %s, %s, %s)"
cursor.executemany(insert_query, album_data)
cnx.commit()
print("album에 데이터가 삽입되었습니다.")

# details_table 데이터 삽입
details_data = details_table_df.values.tolist()
insert_query = "INSERT INTO details (track_id, danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration_ms) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
cursor.executemany(insert_query, details_data)
cnx.commit()
print("details에 데이터가 삽입되었습니다.")

# track_artist_table 데이터 삽입
track_artist_data = track_artist_table_df.values.tolist()
insert_query = "INSERT INTO track_artist (artist_id, track_id) VALUES (%s, %s)"
cursor.executemany(insert_query, track_artist_data)
cnx.commit()
print("track_artist에 데이터가 삽입되었습니다.")

# album_artists_table 데이터 삽입
album_artists_data = album_artists_table_df.values.tolist()
insert_query = "INSERT INTO album_artists (album_id, album_artists_id) VALUES (%s, %s)"
cursor.executemany(insert_query, album_artists_data)
cnx.commit()
print("album_artists에 데이터가 삽입되었습니다.")

# album_tracks_table 데이터 삽입
album_tracks_data = album_tracks_table_df.values.tolist()
insert_query = "INSERT INTO album_tracks (album_id, album_tracks_id) VALUES (%s, %s)"
cursor.executemany(insert_query, album_tracks_data)
cnx.commit()
print("album_tracks에 데이터가 삽입되었습니다.")

# artist_genres_table 데이터 삽입
artist_genres_data = artist_genres_table_df.values.tolist()
insert_query = "INSERT INTO artist_genres (artist_genres, artist_id) VALUES (%s, %s)"
cursor.executemany(insert_query, artist_genres_data)
cnx.commit()
print("artist_genres에 데이터가 삽입되었습니다.")

# 작업 완료 후 연결 종료
cursor.close()
cnx.close()

데이터베이스 'spotify'가 생성되었습니다.
테이블 'track'이(가) 생성되었습니다.
테이블 'artist'이(가) 생성되었습니다.
테이블 'album'이(가) 생성되었습니다.
테이블 'details'이(가) 생성되었습니다.
테이블 'track_artist'이(가) 생성되었습니다.
테이블 'album_artists'이(가) 생성되었습니다.
테이블 'album_tracks'이(가) 생성되었습니다.
테이블 'artist_genres'이(가) 생성되었습니다.
track에 데이터가 삽입되었습니다.
artist에 데이터가 삽입되었습니다.
album에 데이터가 삽입되었습니다.
details에 데이터가 삽입되었습니다.
track_artist에 데이터가 삽입되었습니다.
album_artists에 데이터가 삽입되었습니다.
album_tracks에 데이터가 삽입되었습니다.
artist_genres에 데이터가 삽입되었습니다.
